In [1]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import graph_to_paths
import napari

# Downloading Brain data tutorial
We have prepared a 2 low-resolution brain volumes, as well as a single axon segment, at the below s3 urls (see `uploading_brains.ipynb`).
The method demonstrated below pulls a region of the volume around an annotated axon point set by the user.

## 1) Define Variables
- `mip` ranges from higher resolution (0) to lower resolution (1).
- `v_id` are vertex ids ranging from the soma (0) to the end of the axon (1649).
- `radius` is the radius to pull around the selected point, in voxels.

In [3]:
dir = "s3://mouse-light-viz/precomputed_volumes/brain1"
dir_segments = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
mip = 0
v_id = 0
radius = 75

## 2) Create a NeuroglancerSession instance and download the volume.

In [4]:
# get image and center point
ngl_sess = NeuroglancerSession(mip = mip, url = dir, url_segments=dir_segments)
img, bbox, vox = ngl_sess.pull_voxel(2, v_id, radius)
print(f"\n\nDownloaded volume is of shape {img.shape}, with total intensity {sum(sum(sum(img)))}.")

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.60it/s]
Downloading: 46it [00:07,  6.05it/s]

Downloaded volume is of shape (151, 151, 151), with total intensity 4968390.



## 3) Generate a graph from the segment data within the volume, and convert it to paths.

In [5]:
G_sub = ngl_sess.get_segments(2, bbox)
paths = graph_to_paths(G_sub)
print(f"Selected volume contains {G_sub.number_of_nodes()} nodes and {len(paths)} paths")

Downloading: 100%|██████████| 1/1 [00:00<00:00,  5.05it/s]
Selected volume contains 6 nodes and 2 paths


## 4) View the volume with paths overlaid via napari.

In [6]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)
    viewer.add_points(vox, size=1, opacity=0.5)